In [1]:
from __future__ import absolute_import, division, print_function
import gensim
import tensorflow as tf
import pandas as pd
import sqlalchemy

print('gensim version: \t%s' % gensim.__version__)
print('TensorFlow version: \t%s' % tf.__version__)
print('Pandas version: \t%s' % pd.__version__)
print('SQLalchemy version: \t%s' % sqlalchemy.__version__)

In [2]:
df = spark.sql("select JD_SOLUTIONHTML from edl_current.product_casemanagement_dtacccms_product_casemanagement_dtac_qwsolutionslnhtml AS SOLUTION_DETAILS_FLAT inner join edl_current.product_casemanagement_dtacccms_product_casemanagement_dtac_qwsolution AS PCDTAC_SOLUTIONS ON (SOLUTION_DETAILS_FLAT.QW_SOLUTIONID = PCDTAC_SOLUTIONS.QW_SOLUTIONID) where PCDTAC_SOLUTIONS.DEERE_LANGUAGE_ID = '19' ")

In [3]:
data_file_name = "df"
raw_df = df.toPandas() #convert spark.sql to a pandas data frame
print("Data loaded")

In [6]:

#raw_corpus = raw_df['JD_SOLUTIONHTML']
raw_corpus = raw_corpus.join((str(raw_df['JD_SOLUTIONHTML']+" ")))
print("Raw Corpus contains {0:,} characters".format(len(raw_corpus)))

In [7]:
import nltk
# download the punkt tokenizer
nltk.download('punkt')
print("The punkt tokenizer is downloaded")

In [8]:
tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")
print("The punkt tokenizer is loaded")
# we tokenize the raw string into raw sentences
raw_sentences = tokenizer.tokenize(raw_corpus)
print("We have {0:,} raw sentences".format(len(raw_sentences)))

In [9]:
import re
def clean_and_split_str(string):
    strip_special_chars = re.compile("[^A-Za-z]+")
    string = re.sub(strip_special_chars, " ", string)
    return string.strip().split()

In [10]:
sentences = []
for raw_sent in raw_sentences:
    if len(raw_sent) > 0:
        sentences.append(clean_and_split_str(raw_sent))
print("We have {0:,} clean sentences".format(len(sentences)))

In [11]:
print(raw_sentences[5])
print()
print(sentences[5])

In [12]:
token_count = sum([len(sentence) for sentence in sentences])
print("The dataset corpus contains {0:,} tokens".format(token_count))

In [13]:
import multiprocessing

#Dimensionality of the resulting word vectors
num_features = 300

#Minimum word count threshold
min_word_count = 3

#Number of threads to run in parallel
num_workers = multiprocessing.cpu_count()

#Context window length
context_size = 7

#Seed for the RNG, to make the result reproducible
seed = 3

In [14]:
import gensim
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
word2vec_model = gensim.models.word2vec.Word2Vec(sg=1,
    seed=seed,
    workers=num_workers, 
    size=num_features, 
    min_count=min_word_count, 
    window=context_size)

In [15]:
word2vec_model.build_vocab(sentences=sentences)

print("The vocabulary is built")
print("Word2Vec vocabulary length: ", len(word2vec_model.wv.vocab))
print(word2vec_model.wv.vocab)

In [16]:
#Start training the model

word2vec_model.train(sentences, total_examples = token_count, epochs = 50)
print("Training finished")

In [18]:
word2vec_model.wv.most_similar(positive=["SPEED"])

In [19]:
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
model.save(os.path.join(MODEL_DIR,'word2vec'))